In [106]:
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import numpy as np
from nltk.stem import WordNetLemmatizer
from profanity_check import predict, predict_prob
from nltk.tokenize.toktok import ToktokTokenizer
import csv

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\binta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\binta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [107]:
def cleaning_url(raw_url)->str:
    url_ = re.sub(';|,| |-|/|:|\.',' ',raw_url)
    # url_ = re.sub('[;,-/:\.]',' ',url[0])
    url_ = url_.lower()
    url_ = re.sub(r'https|com|net|co|us|fr|uk|id|care|http|www', ' ', url_)
    return url_
#aman

In [108]:
# #test url cleaning
# text = "https://stackoverflow.com/questions/55407843/how-to-handle-url-links-in-text-data-while-preprocessing-data-in-nlp"
# cleaning_url(text)

In [109]:
#test stem vs lemmatize
# stemmer = PorterStemmer()
# lemmatizer = WordNetLemmatizer()
# def stem_vs_lemmatize(text):
#     print(stemmer.stem(text))
#     print(lemmatizer.lemmatize(text))
# stem_vs_lemmatize('cry')
# stem_vs_lemmatize('handler')
# stem_vs_lemmatize("dont")


In [110]:
stopwords_english = stopwords.words('english')
stemmer = PorterStemmer()
# lemmatizer = WordNetLemmatizer()

#its yes no memukul -> pukul,  4 -> 100

mtokenize = ToktokTokenizer()

# stopword tambahan
more_stopword = set([])

# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
])

# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)

def cleaning_text(raw_text)->str:
    '''
    :rtype: string
    :type text : string
    :param text: type string, text mentah
    :return: type string, text yang sudah dibersihkan
    '''
    # print("-----------------------------------------------------------------")
    text_ = re.sub(r'\n', ' ', raw_text)  # menghapus baris baru
    # print("sebelum : "+text_)
    text_ = text_.strip(" ")  # menghapus karakter spasi pada awal dan akhir kalimat
    # print(text_)
    symbols_regrex_pattern = re.compile(pattern="["
                                                u"\U0001F600-\U0001F64F"  # emoticons
                                                u"\U0001F300-\U0001F5FF"  # simbol & piktograf
                                                u"\U0001F680-\U0001F6FF"  # simbol transport & map
                                                "]+", flags=re.UNICODE)    
    text_ = symbols_regrex_pattern.sub(r' ', text_)  
    # print(text_)
    text_ = text_.lower() 
    # print(text_)
    text_ = re.sub(r'^https?:\/\/.*[\r\n]*', ' ', text_, flags=re.MULTILINE)  # menghapus hyperlink
    # print(text_)
    text_ = re.sub('[0-9]+', ' ', text_)  # menghapus angka
    # print(text_)
    tokens = word_tokenize(text_)  # tokenize kata-kata
    tokens = mtokenize.tokenize(text_)  # tokenize kata-kata
    # print(tokens)

    text_clean = []

    for word in tokens:
        # print(word)
        if (word not in stopwords_english and  # menghilangkan stopwords. proses filtering
                word not in emoticons and  # menghilangkan emoticons. proses filtering
                word not in more_stopword and
                word not in string.punctuation):  # menghilangkan tanda baca. proses filtering
            # stem_word = lemmatizer.lemmatize(word)
            # print(word)
            stem_word = stemmer.stem(word)  # mengubah ke kata dasar. proses stemming
            # print(stem_word)
            text_clean.append(stem_word)

    text_clean = " ".join(text_clean)
    # print(text_clean)
    text_clean = re.sub(r'[?\|$\.!_:")(\+,\*&%#@`\'\"=\\/]', ' ', text_clean)  # menghapus beberapa karakter khusus
    # print(text_clean)
    text_clean = re.sub(' +', ' ', text_clean)  # lebih dari satu spasi yang berdekatan, menjadi satu spasi
    text_ = text_.strip(" ")  # menghapus karakter spasi pada awal dan akhir kalimat

    # print("sesudah : ", text_clean)
    # print("-----------------------------------------------------------------")
    return text_clean
#aman


In [111]:
#test url cleaning
# text = "https://stackoverflow.com/questions/55407843/how-to-handle-url-links-in-text-data-while-preprocessing-data-in-nlp"
# text = 'python - "UnboundLocalError: local variable referenced before assignment" after an if statement - Stack Overflow'
# text = """I have also tried searching for the answer but I don't understand the answers to other people's similar problems...
# tfile= open("/home/path/to/file",'r') 
# def temp_sky(lreq, breq):
#     for line in """
# cleaning_text(text)

In [112]:
forbidden_word = []
with open('data/bad-bad-word-cleaned.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
for i in data:
    forbidden_word.append(i[0])

# print(forbidden_word)

def check_forbidden_word(raw_text)->int:
    '''
    :rtype: integer
    :type text : string
    :param text: type string, text 
    :return: integer, 1 jika menemukan forbidden word, 0 jika tidak ditemukan forbidden word
    '''    
    text_ = raw_text.lower()
    for fword in forbidden_word:
        if fword in text_:
            return 1
    return 0
#aman

In [113]:
# text = "Pornhub is the world’s leading free porn site. Choose from millions of hardcore videos that stream quickly and in high quality, including amazing VR Porn. The largest adult site on the Internet just keeps getting better. We have more pornstars and real amateurs than anyone else. It’s fast, it’s free, it’s Pornhub!"
# text = "PORN"
# check_forbidden_word(text)

In [114]:
def check_ip_domain(raw_url)->int:
    url_ = re.sub(r'https|http|/|:', ' ', raw_url)
    url_ = re.sub(' +', ' ', url_)
    url_ = url_.strip(" ")
    url_ = url_.split(" ")
    # print(url_[0].split('.'))
    # print(url_[0].split('.')[-1])
    if not url_[0].split('.')[-1].isalpha() :
        return 1
    else :
        return 0
#aman        

In [115]:
# text = "http://192.158.1.38/safely-collecting-hot-flows-from-android-native-ui-f22f645edb44"
# text = "http://0xC0.0x00.0x02.0xEB/safely-collecting-hot-flows-from-android-native-ui-f22f645edb44"
# text = 'https://stackoverflow.com/questions/30626182/determine-if-host-is-domain-name-or-ip-in-python'
# text = ""
# check_ip_domain(text)

In [116]:
def check_profanity_check_lib(text)->int:
    # print(predict_prob([text]))
    # print(predict([text]))
    # if predict_prob([text])[0] > 0.05:
    #     return 1
    return predict([text])[0]
#aman

In [117]:
# text = "Pornhub is the world’s leading free porn site. Choose from millions of hardcore videos that stream quickly and in high quality, including amazing VR Porn. The largest adult site on the Internet just keeps getting better. We have more pornstars and real amateurs than anyone else. It’s fast, it’s free, it’s Pornhub!"
# text = "python unboundlocalerror local variabl referenc assign statement stack overflow"
# text = "Free Porn, Sex, Tube Videos, XXX Pics, Pussy in Porno Movies - XNXX.COM"
# check_profanity_check_lib(text)


In [118]:
def profanity_check_rating(raw_rating):
    rating_ = raw_rating.lower()
    forbidden_rating = ["mature", "adult", "restricted","RTA-5042-1996-1400-1577-RTA"]
    forbidden_rating = [fr.lower() for fr in forbidden_rating]
    for fr in forbidden_rating:
        if fr in rating_:
            return 1
    return 0
#aman

In [119]:
def matrix_profanity_check_rating(raw_rating):
    rating_ = raw_rating.lower()
    forbidden_rating = ["mature", "adult", "restricted","RTA-5042-1996-1400-1577-RTA"]
    forbidden_rating = [fr.lower() for fr in forbidden_rating]
    for fr in forbidden_rating:
        if fr in rating_:
            return {"rating" : 1}

    return {"rating" : 0}
#aman

In [120]:
# test = "RTA-5042-1996-1400-1577-RTA"
# profanity_check_rating(test)

In [121]:
def profanity_check_url(raw_url):
    url_clean = cleaning_url(raw_url)
    text_ = cleaning_text(url_clean)

    v_check_forbidden_word = check_forbidden_word(url_clean)
    if v_check_forbidden_word == 1 : return 1

    v_check_profanity_check_lib = check_profanity_check_lib(text_)
    if v_check_profanity_check_lib == 1 : return 1

    v_check_ip_domain = check_ip_domain(raw_url)
    if v_check_ip_domain == 1 : return 1
    
    return 0

In [122]:
def matrix_profanity_check_url(raw_url):
    url_clean = cleaning_url(raw_url)
    text_ = cleaning_text(url_clean)

    v_check_forbidden_word = check_forbidden_word(url_clean)
    v_check_profanity_check_lib = check_profanity_check_lib(text_)
    v_check_ip_domain = check_ip_domain(raw_url)
    
    return {"fword" : v_check_forbidden_word,"proflib" : v_check_profanity_check_lib,"ipdomain" : v_check_ip_domain}

In [123]:
def profanity_check_text(raw_text)->int:
    text_ = cleaning_text(raw_text)

    v_check_forbidden_word = check_forbidden_word(raw_text)
    if v_check_forbidden_word == 1 : return 1

    v_check_profanity_check_lib = check_profanity_check_lib(text_)
    if v_check_profanity_check_lib == 1 : return 1

    return 0


In [133]:
def matrix_profanity_check_text(raw_text)->int:
    text_ = cleaning_text(raw_text)

    v_check_forbidden_word = check_forbidden_word(raw_text)
    v_check_profanity_check_lib = check_profanity_check_lib(text_)

    return {"fword" : v_check_forbidden_word,"proflib" : v_check_profanity_check_lib}

In [125]:
def profanity_check_overall(url, title, description, keyword, rating):
    v_profanity_check_url = profanity_check_url(url)
    if v_profanity_check_url == 1 : return 1

    v_profanity_check_title = profanity_check_text(title)
    if v_profanity_check_title == 1 : return 1

    v_profanity_check_description = profanity_check_text(description)
    if v_profanity_check_description == 1 : return 1

    v_profanity_check_keyword = profanity_check_text(keyword)
    if v_profanity_check_keyword == 1 : return 1

    v_profanity_check_rating = profanity_check_rating(rating)
    if v_profanity_check_rating == 1 : return 1

    return 0


In [134]:
def matrix_profanity_check_overall(url, title, description, keyword, rating):
    v_profanity_check_url = matrix_profanity_check_url(url)
    v_profanity_check_title = matrix_profanity_check_text(title)
    v_profanity_check_description = matrix_profanity_check_text(description)
    v_profanity_check_keyword = matrix_profanity_check_text(keyword)
    v_profanity_check_rating = matrix_profanity_check_rating(rating)

    return {"url" : v_profanity_check_url , "title" : v_profanity_check_title ,
     "description" : v_profanity_check_description , "keyword" : v_profanity_check_keyword ,
      "rating" : v_profanity_check_rating }

In [135]:
# url1 = 
# title1 = 
# desc1 =
# rating =
# keyword =

# url1 = "https://stackoverflow.com/questions/15367760/unboundlocalerror-local-variable-referenced-before-assignment-after-an-if-stahttps://stackoverflow.com/questions/15367760/unboundlocalerror-local-variable-referenced-before-assignment-after-an-if-sta"
# title1 = 'python - "UnboundLocalError: local variable referenced before assignment" after an if statement - Stack Overflow'
# desc1 = """I have also tried searching for the answer but I don't understand the answers to other people's similar problems...

# tfile= open("/home/path/to/file",'r') 

# def temp_sky(lreq, breq):
#     for line in """

title1 = "Free Porn, Sex, Tube Videos, XXX Pics, Pussy in Porno Movies - XNXX.COM"
url1 = "https://www.xnxx.com/"
desc1 = "XNXX delivers free sex movies and fast free porn videos (tube porn). Now 10 million+ sex vids available for free! Featuring hot pussy, sexy girls in xxx rated porn clips."
rating1 = ""
keyword1 = ""

# profanity_check_overall("pornhub.com","pornhub","Pornhub is the world’s leading free porn site. Choose from millions of hardcore videos that stream quickly and in high quality, including amazing VR Porn. The largest adult site on the Internet just keeps getting better. We have more pornstars and real amateurs than anyone else. It’s fast, it’s free, it’s Pornhub!")
matrix_profanity_check_overall(url1,title1,desc1,rating1,keyword1)

{'url': {'fword': 0, 'provlib': 0, 'ipdomain': 0},
 'title': {'fword': 1, 'provlib': 1},
 'description': {'fword': 1, 'provlib': 1},
 'keyword': {'fword': 0, 'provlib': 0},
 'rating': {'rating': 0}}